# RTS Polygon Bounding Box Retrieval
### This script extracts buffered bounding boxes from the RTS polygon dataset in order to define the AOIs for planet data download.
### TODO:
- Update polygon input when the new version is finalized!

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap

In [ ]:
# Import polygons
rts = ee.FeatureCollection('users/hrodenhizer-woodwellclimate/fixed_polygons_for_Yili_Apr_2022');

print(rts.size().getInfo())

In [ ]:
# Set up map
Map = geemap.Map();
Map.centerObject(rts);

Map.addLayer(rts, {'color': 'FF0000'}, 'RTS')

In [ ]:
# Extract bounding boxes
def extractBBox(feature):
    return feature.bounds().buffer(1500).bounds()

rts_bboxes = rts.map(extractBBox)

Map.addLayer(rts_bboxes, {'color': '000000'}, 'BBoxes')

In [ ]:
rts_union = rts_bboxes.union(0.1)
Map.addLayer(rts_union, {'color': 'FFFF00'}, 'Union')

In [ ]:
Map

In [ ]:
# lots of overlap in bounding boxes
def setArea(feature):
    return feature.set('area', ee.Number(feature.area(1)).divide(ee.Number(10e5))); # return area in km^2

rts_bboxes = rts_bboxes.map(setArea)

rts_bboxes.aggregate_sum('area').getInfo()

In [ ]:
# taking the union results in about 1/3 the area
rts_union = rts_union.map(setArea)

rts_union.aggregate_sum('area').getInfo()

In [ ]:
# export output to google drive
task = ee.batch.Export.table.toDrive(
    collection = rts_union,
    description = 'RTS_buffer',
    folder = 'Earth Engine Exports',
    fileFormat = 'SHP'
)
task.start()